In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time
import sys
from sklearn.svm import SVC
import numpy as np
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, precision_score, f1_score, recall_score, roc_auc_score
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
# Load the data
data = pd.read_csv('train/train_radiology.csv')

In [4]:
data.drop(columns=['charttime','note_type','note_seq'],inplace=True)

In [5]:
data

,patient_id,text
0,p100001,"AP CHEST, 7:27 A.M., ___. HISTORY: ___ man w..."
1,p100001,INDICATION: ___ year old man with pneumonia a...
2,p100002,"EXAMINATION: CTA CHEST WANDW/O CANDRECONS, NO..."
3,p100004,EXAMINATION: Left vertebral artery angiogram....
4,p100004,EXAMINATION: CTA HEAD AND CTA NECK Q16 CT IN...
...,...,...
27917,p117132,EXAMINATION: CHEST (PORTABLE AP) INDICATION:...
27918,p117132,EXAMINATION: CHEST (PORTABLE AP) INDICATION:...
27919,p117132,EXAMINATION: CHEST (PORTABLE AP) INDICATION:...
27920,p117132,EXAMINATION: FEMORAL VASCULAR US LEFT PORT I...


In [4]:
# Group radiology reports by patient_id and concatenate the text
data = data.groupby('patient_id')['text'].apply(lambda x: ' '.join(x)).reset_index()

In [5]:
# Merge radiology data with label data based on patient_id
data = pd.merge(data, labels, on='patient_id', how='left')

In [6]:
# Tokenization
data['tokenized_text'] = data['text'].apply(lambda x: word_tokenize(x.lower()))

In [7]:
# Data Preprocessing
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

In [8]:
def preprocess_text(tokens):
    tokens = [token for token in tokens if token.isalpha()]  # Remove punctuation
    tokens = [token for token in tokens if token not in stop_words]  # Remove stop words
    tokens = [ps.stem(token) for token in tokens]  # Stemming
    return tokens

In [9]:
data['tokenized_text'] = data['tokenized_text'].apply(preprocess_text)

In [15]:
data['tokenized_text']

0        [ap, chest, histori, man, rapidli, develop, ba...
1        [examin, cta, chest, candrecon, indic, histori...
2        [examin, left, vertebr, arteri, angiogram, rig...
3        [indic, man, histori, copd, pneumonia, report,...
4        [portabl, chest, comparison, studi, earlier, d...
                               ...                        
13703    [ap, chest, histori, acut, pancreat, question,...
13704    [examin, portabl, chest, radiograph, indic, ye...
13705    [examin, chest, portabl, ap, indic, year, old,...
13706    [examin, chest, portabl, ap, indic, year, old,...
13707    [examin, cta, chest, indic, lung, cancer, dysp...
Name: tokenized_text, Length: 13708, dtype: object

In [39]:
# Feature Extraction
vectorizer = CountVectorizer(max_features=32)
X = vectorizer.fit_transform(data['tokenized_text'].apply(lambda x: ' '.join(x)))

In [17]:
X

<13708x512 sparse matrix of type '<class 'numpy.int64'>'
	with 1174069 stored elements in Compressed Sparse Row format>

In [30]:
Y = data['label']

In [40]:
# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate on validation set
val_predictions = model.predict_proba(X_val)

In [32]:
val_predictions

array([[0.95, 0.05],
       [0.96, 0.04],
       [0.86, 0.14],
       ...,
       [0.87, 0.13],
       [0.92, 0.08],
       [0.9 , 0.1 ]])

In [41]:
val_predictions = (val_predictions[:, 1] > 0.5).astype(int)

In [42]:
val_accuracy = accuracy_score(y_val, val_predictions)
print("Validation Accuracy:", val_accuracy)

# Get feature importance
feature_importance = model.feature_importances_

# Map feature names to their importance scores
feature_names = vectorizer.get_feature_names_out()
importance_dict = dict(zip(feature_names, feature_importance))

# Sort feature importance dictionary by importance score (descending order)
sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print top features
top_features = 10  # Number of top features to display
print(f"Top {top_features} features:")
for feature, importance in sorted_importance[:top_features]:
    print(f"{feature}: {importance}")


Validation Accuracy: 0.8915223336371924
Top 10 features:
right: 0.05343941350213106
left: 0.04859303789060004
tube: 0.041319961712764176
chest: 0.039303128140130074
lung: 0.03738167523182705
find: 0.03669553473289732
bilater: 0.03592409726177108
edema: 0.03503992751769627
pulmonari: 0.032780770762557546
evid: 0.032730274530945715
